# Web Scraping Le Matin

## Installation du package selenium et du chromedriver

In [ ]:
# installation du package selenium
!pip install selenium
# installation du chormedriver
!apt install chromium-chromedriver
# copie du chromedriver
!cp C:\Users\Adib\Documents\Stage\Stage 1A\Selenium\chromedriver.exe

## Importation des librairies nécessaires

In [89]:
from selenium import webdriver
import pandas as pd

## Modification des options du chromedriver

In [90]:
# options chromedriver pour l'utiliser dans un notebook
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

## Instanciation du WebDriver de Chrome

In [91]:
# création de l'instance WebDriver de Chrome
driver = webdriver.Chrome('chromedriver', options = options)
# lien du site Le Matin
website = 'https://lematin.ma/'

## Liste des catégories du site

In [92]:
# catégories du site Le Matin
categories = ['activites-royales', 'nation', 'economie', 'monde', 'societe', 'culture', 'regions', 'emploi', 'sports', 'automobile']

## Récupération des liens d'articles

In [93]:
# liste des liens vers les articles
articles_links = []
# parcours des catégories
for category in categories:
  category_link = website + 'journal/' + category
  # parcours des pages de chaque catégorie
  page_number = 0
  while (page_number < 3):
    webpage_link = category_link + '/' + str(page_number)
    page_number += 1
    # parcours des articles de chaque page
    driver.get(webpage_link)
    href_links = driver.find_elements_by_xpath('//div/div/div/div/a[@href]')
    for href_link in href_links:
      articles_links.append(href_link.get_attribute("href"))
articles_links = list(filter(lambda article_link: 'https://lematin.ma/express' in article_link, articles_links))

## Vérification de la liste de liens

In [94]:
# affichage de 5 liens pris au hasard
articles_links[95:100]

['https://lematin.ma/express/2022/nasser-bourita-voici-maroc-me-bataille-co-developpement-afrique/376926.html',
 'https://lematin.ma/express/2022/maroc-afrique-sud-victoire-bonne-moral-mais-entretient-doutes/376923.html',
 'https://lematin.ma/express/2022/abdellatif-hammouchi-recoit-dg-police-federale-allemagne/376921.html',
 'https://lematin.ma/express/2022/arabie-saoudite-munir-mohamedi-debarque-al-wehda/376919.html',
 'https://lematin.ma/express/2022/maroc-afrique-sud-impressions-mitigees-malgre-victoire-face-aux-bafanas-bafanas/376913.html']

In [95]:
# affichage du nombre total de liens
len(articles_links)

1080

## Récupération du contenu des articles

In [97]:
# data frame des données collectés
df_articles = pd.DataFrame()
# parcours des liens collectés
for article_link in articles_links[:500]:
  # accès à la page de l'article
  driver.get(article_link)
  # récupération du titre de l'article
  title = driver.find_element_by_id('title').text
  # récupération de l'auteur de l'article
  author = driver.find_element_by_class_name('author').text.split(',')[0]
  # récupération de la date de publication de l'article
  date = driver.find_element_by_xpath('//time').text
  # récupération du contenu de l'article
  content_scrap = driver.find_elements_by_xpath('//div/p')
  content = [p.text for p in content_scrap if len(p.text) > 50]
  content = ' '.join(content)
  # création de la liste de nos données
  article = {'title' : title, 'author' : author, 'date' : date, 'content' : content}
  # transformation en data frame
  df_article = pd.DataFrame(article, index = [0])
  # transformation en data frame
  df_articles = df_articles.append(df_article, ignore_index=True)

## Vérification des données collectés

In [98]:
# affichage des premières valeurs
df_articles.head()

,title,author,date,content
0,Naissance dans le foyer de S.A.R. le Prince Mo...,LE MATIN 02 juin 2022 à 06:37 3875,02 juin 2022 à 06:37,"Le porte-parole du Palais Royal a annoncé, jeu..."
1,S.M. le Roi adresse un message de félicitation...,LE MATIN 30 mai 2022 à 23:43 2014,30 mai 2022 à 23:43,"Sa Majesté le Roi Mohammed VI, que Dieu L'assi..."
2,Ligue des Champions : S.M. le Roi félicite dan...,LE MATIN 30 mai 2022 à 22:30,30 mai 2022 à 22:30,"Sa Majesté le Roi Mohammed VI, que Dieu L'assi..."
3,Message de condoléances et de compassion de S....,MAP,27 mai 2022 à 10:18,Sa Majesté le Roi Mohammed VI a adressé un mes...
4,S.M. le Roi adresse un message de félicitation...,MAP,27 mai 2022 à 10:16,Sa Majesté le Roi Mohammed VI a adressé un mes...


In [99]:
# affichage de la taille de ntore data frame
len(df_articles)

500

## Conversion en fichier csv

In [100]:
# transformation du data frame en fichier csv
df_articles.to_csv('le_matin_articles.csv')